In [363]:
import pandas as pd
import re
FICHEROS_CSV = [
            'data_dictionary', 'order_details',
            'orders', 'pizza_types', 'pizzas']

In [364]:
'''i = 3

df = pd.read_csv(FICHEROS_CSV[i] + '.csv', sep=',', encoding = 'cp1252')
string = df.dtypes.to_string()
mensaje = '\nEl fichero ' + FICHEROS_CSV[i]
mensaje += '.csv contiene las siguientes columnas:\n'
mensaje += '\n' + string +'\n'
file = open('analisis_datos.txt', 'w')
file.write(mensaje)
file.close()'''

"i = 3\n\ndf = pd.read_csv(FICHEROS_CSV[i] + '.csv', sep=',', encoding = 'cp1252')\nstring = df.dtypes.to_string()\nmensaje = '\nEl fichero ' + FICHEROS_CSV[i]\nmensaje += '.csv contiene las siguientes columnas:\n'\nmensaje += '\n' + string +'\n'\nfile = open('analisis_datos.txt', 'w')\nfile.write(mensaje)\nfile.close()"

In [365]:
'''string = df.dtypes.to_dict()
for key in string.keys():
    string[key] = str(string[key])
print(string)
dicc = {FICHEROS_CSV[i] + '.csv':string}
print(dicc)
df2 = pd.DataFrame(dicc)
df2'''

"string = df.dtypes.to_dict()\nfor key in string.keys():\n    string[key] = str(string[key])\nprint(string)\ndicc = {FICHEROS_CSV[i] + '.csv':string}\nprint(dicc)\ndf2 = pd.DataFrame(dicc)\ndf2"

In [366]:
'''file = open('analisis_datos.txt', 'w')
file.close()

for archivo in FICHEROS_CSV:

    df = pd.read_csv(archivo + '.csv', sep=',', encoding = 'cp1252')

    mensaje = '\nEl fichero ' + archivo
    mensaje += '.csv contiene las siguientes columnas:\n\n'

    for colum in df.columns:
        tmp = colum
        tmp += '    ' + str(df[colum].dtype)
        tmp += '    Valores Null/Nan = ' + str(df[colum].isnull().sum())
        mensaje += tmp + '\n'

    file = open('analisis_datos.txt', 'a')
    file.write(mensaje)
    file.close()
'''

"file = open('analisis_datos.txt', 'w')\nfile.close()\n\nfor archivo in FICHEROS_CSV:\n\n    df = pd.read_csv(archivo + '.csv', sep=',', encoding = 'cp1252')\n\n    mensaje = '\nEl fichero ' + archivo\n    mensaje += '.csv contiene las siguientes columnas:\n\n'\n\n    for colum in df.columns:\n        tmp = colum\n        tmp += '    ' + str(df[colum].dtype)\n        tmp += '    Valores Null/Nan = ' + str(df[colum].isnull().sum())\n        mensaje += tmp + '\n'\n\n    file = open('analisis_datos.txt', 'a')\n    file.write(mensaje)\n    file.close()\n"

In [367]:
df_pedidos = pd.read_csv('order_details.csv', sep=',', encoding='cp1252')
df_tipos = pd.read_csv('pizzas.csv', sep=',', encoding='cp1252')
df_pizzas = df_pedidos.groupby(df_pedidos['pizza_id']).sum()
#df_pizzas.sort_index()

In [368]:
df_pizzas_total = df_pizzas.merge(df_tipos, how='inner', left_on='pizza_id', right_on='pizza_id')
df_pizzas_total = df_pizzas_total[['pizza_type_id', 'size', 'quantity']]
df_pizzas_total

,pizza_type_id,size,quantity
0,bbq_ckn,L,992
1,bbq_ckn,M,956
2,bbq_ckn,S,484
3,big_meat,S,1914
4,brie_carre,S,490
...,...,...,...
86,the_greek,XL,552
87,the_greek,XXL,28
88,veggie_veg,L,427
89,veggie_veg,M,635


In [369]:
df_ingredientes_pizzas = pd.read_csv('pizza_types.csv', sep=',', encoding='cp1252')
df_ingredientes_pizzas = df_ingredientes_pizzas[['pizza_type_id', 'ingredients']]
df_pizzas_final = df_pizzas_total.merge(df_ingredientes_pizzas, how='inner', left_on='pizza_type_id', right_on='pizza_type_id')


In [370]:
def str_to_list(string: str):

    if isinstance(string, str):
        salsa = 0
        queso = 0
        if not re.search('Sauce', string):
            salsa = 1
        if not re.search('Mozzarella Cheese', string):
            queso = 1
        string = string.split(', ')
        if salsa:
            string.append('Tomato Sauce')
        if queso:
            string.append('Mozarrella Cheese')
    return string

df_pizzas_final['ingredients'] = df_pizzas_final['ingredients'].apply(str_to_list)


In [371]:
dicc = {'pizza_type_id': [], 'size': []}
ingredientes = []
for i in df_pizzas_final.index:
    for j in range(len(df_pizzas_final['ingredients'][i])):
        if df_pizzas_final['ingredients'][i][j] not in ingredientes:
            ingredientes.append(df_pizzas_final['ingredients'][i][j])
            dicc[df_pizzas_final['ingredients'][i][j]] = []
sizes = {'S': 1, 'M': 1.5, 'L': 2,'XL': 2.5, 'XXL': 3}
# print(ingredientes)
for i in df_pizzas_final.index:
    pizza = df_pizzas_final['pizza_type_id'][i]
    size = df_pizzas_final['size'][i]
    dicc['pizza_type_id'].append(pizza)
    dicc['size'].append(size)
    for ingrediente in ingredientes:
        if ingrediente in df_pizzas_final['ingredients'][i]:
            
            dicc[ingrediente] += [sizes[size]]
        elif ingrediente not in df_pizzas_final['ingredients'][i]:
            dicc[ingrediente] += [0]
df_pizzas_final.pop('ingredients')
df_tmp = pd.DataFrame(dicc)
df_tmp

,pizza_type_id,size,Barbecued Chicken,Red Peppers,Green Peppers,Tomatoes,Red Onions,Barbecue Sauce,Mozarrella Cheese,Bacon,...,Genoa Salami,Prosciutto di San Daniele,Arugula,Coarse Sicilian Salami,Luganega Sausage,Onions,Soppressata Salami,Peperoncini verdi,Thai Sweet Chilli Sauce,Beef Chuck Roast
0,bbq_ckn,L,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,bbq_ckn,M,1.5,1.5,1.5,1.5,1.5,1.5,1.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,bbq_ckn,S,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,big_meat,S,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,brie_carre,S,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,the_greek,XL,0.0,0.0,0.0,2.5,2.5,0.0,2.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5
87,the_greek,XXL,0.0,0.0,0.0,3.0,3.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
88,veggie_veg,L,0.0,2.0,2.0,2.0,2.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89,veggie_veg,M,0.0,1.5,1.5,1.5,1.5,0.0,1.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [372]:
df_pizzas_final = df_pizzas_final.merge(df_tmp, how='inner', left_on=['pizza_type_id', 'size'], right_on=['pizza_type_id', 'size'])
# df_pizzas_final = df_pizzas_final[df_pizzas_final.where(df_pizzas_final['size_x']==df_pizzas_final['size_y'])]
df_pizzas_final


,pizza_type_id,size,quantity,Barbecued Chicken,Red Peppers,Green Peppers,Tomatoes,Red Onions,Barbecue Sauce,Mozarrella Cheese,...,Genoa Salami,Prosciutto di San Daniele,Arugula,Coarse Sicilian Salami,Luganega Sausage,Onions,Soppressata Salami,Peperoncini verdi,Thai Sweet Chilli Sauce,Beef Chuck Roast
0,bbq_ckn,L,992,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,bbq_ckn,M,956,1.5,1.5,1.5,1.5,1.5,1.5,1.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,bbq_ckn,S,484,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,big_meat,S,1914,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,brie_carre,S,490,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,the_greek,XL,552,0.0,0.0,0.0,2.5,2.5,0.0,2.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5
87,the_greek,XXL,28,0.0,0.0,0.0,3.0,3.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
88,veggie_veg,L,427,0.0,2.0,2.0,2.0,2.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89,veggie_veg,M,635,0.0,1.5,1.5,1.5,1.5,0.0,1.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
